
# Preliminaries

the demod operation is, in general:

$$
d = \frac{1}{T}\int_0^T\left[ ADC_1(w_{1c}\cos(\omega t) + w_{1s}\sin(\omega t)) + ADC_2(w_{2c}\cos(\omega t) + w_{2s}\sin(\omega t)) \right]
$$

where $ADC_1$ and $ADC_2$ are linear combinations of $\cos(\omega t)$ and $\sin(\omega t)$. This means that in general it is advantageous for us to denote:

$$
\ket{c} = \cos(\omega t) \\
\ket{s} = \sin(\omega t)
$$

and 

$$
\bra{c} = \frac{1}{T}\int_0^T \cos(\omega t) \\
\bra{s} = \frac{1}{T}\int_0^T \sin(\omega t)
$$

we see that this defines an inner product, which in general is orthogonal *only when $\omega T = 2\pi$. However, we have:

$$
\braket{c|c} + \braket{s|s} = 1 \\
\braket{c|s} - \braket{s|c} = 0
$$

which will be useful later on.

## Calculating $d_I$ and $d_Q$

Since we are in general dealing with two channels, it is useful to rewrite $D$ as

$$
d =
\begin{pmatrix}
\bra{c}& \bra{s}
\end{pmatrix}

\begin{pmatrix}
w_{1c} & w_{2c} \\
w_{1s} & w_{2s} \\
\end{pmatrix}

\begin{pmatrix}
ADC_1\\
ADC_2
\end{pmatrix}
 
= 
\begin{pmatrix}
\bra{c}& \bra{s}
\end{pmatrix}

W

\begin{pmatrix}
ADC_1\\
ADC_2
\end{pmatrix}

$$

The signal that arrives to the ADCs is, without loss of generality:

$$
ADC_1 = \cos(\omega t + \phi + \delta) I + \sin(\omega t + \phi + \delta) Q \\
ADC_2 = -g\sin(\omega t + \phi) I + g\cos(\omega t + \phi) Q \\
$$

which can be written as the vector

$$
\begin{pmatrix}
ADC_1\\
ADC_2
\end{pmatrix}
=
\begin{pmatrix}
cos\delta & \sin\delta \\
0 & g
\end{pmatrix}
\begin{pmatrix}
I & Q \\
Q & -I 
\end{pmatrix}

\begin{pmatrix}
\cos\phi & \sin\phi \\
-\sin\phi & \cos\phi 
\end{pmatrix}
\begin{pmatrix}
\ket{c}\\
\ket{s}
\end{pmatrix}
 
=

C^{-1} (IZ + QX) F \begin{pmatrix}
\ket{c}\\
\ket{s}
\end{pmatrix}
$$

where $Z$ is the Pauli Z matrix and $X$ is the Pauli X matrix. So we need to choose

$$
W_I = F^{-1}ZC \\
W_Q = F^{-1}XC \\
$$

to get $I$ and $Q$, respectively, , because then we have:

$$
d_I = I(\braket{c|c} + \braket{s|s}) + Q(\braket{c|s} - \braket{s|c}) = I \\
d_Q = Q(\braket{c|c} + \braket{s|s}) - I(\braket{c|s} - \braket{s|c}) = Q
$$

# Example


In [4]:
import numpy as np

# Test the demodulation of a signal with a phase and gain mismatch
def calc_demod(I, Q, omega, phi, delta, g):
    t = np.linspace(0, 10, 1000)

    adc1 = np.cos(omega*t + phi + delta)*I + np.sin(omega*t + phi + delta)*Q
    adc2 = -g*np.sin(omega*t + phi)*I + g*np.cos(omega*t + phi)*Q

    X = np.array([[0, 1], [1, 0]])
    Z = np.array([[1, 0], [0, -1]])
    Cinv = np.array([[np.cos(delta), np.sin(delta)], [0, g]])
    C = np.linalg.inv(Cinv)
    Finv = np.array([[np.cos(phi), np.sin(phi)], [-np.sin(phi), np.cos(phi)]])

    W_I = Finv@Z@C
    W_Q = Finv@X@C

    d_I = 0.1*np.trapz(adc1*(W_I[0,0]*np.cos(omega*t) + W_I[1,0]*np.sin(omega*t))
                       + adc2*(W_I[0,1]*np.cos(omega*t) + W_I[1,1]*np.sin(omega*t)), t)

    d_Q = 0.1*np.trapz(adc1*(W_Q[0,0]*np.cos(omega*t) + W_Q[1,0]*np.sin(omega*t))
                         + adc2*(W_Q[0,1]*np.cos(omega*t) + W_Q[1,1]*np.sin(omega*t)), t)

    return d_I, d_Q


for i in range(100):
    I = np.random.rand() - 0.5
    Q = np.random.rand() - 0.5
    omega = 2*np.pi*(np.random.rand()+1)
    g = 0.8 + 0.2*np.random.rand()
    phi = 2*np.pi*np.random.rand()
    delta = 0.1*(np.random.rand()-0.5)

    d_I, d_Q = calc_demod(I, Q, omega, phi, delta, g)
    assert np.allclose(d_I, I, rtol=1e-6)
    assert np.allclose(d_Q, Q, rtol=1e-6)
